# Script tính toán Tầm ảnh hưởng (Baseline 1 – Influence Function truyền thống) cho các láng giềng k-NN trên tập test ParaVE và GramVar.


## Mục đích

- Cài đặt và tính toán **điểm ảnh hưởng truyền thống** (Baseline 1) giữa:
  - Mẫu test (`z_ts`).
  - Các mẫu train láng giềng k-NN (`z_tr`) đã được chọn từ bước k-NN trước đó.
- Sử dụng công thức xấp xỉ Influence Function truyền thống:
  - `IF' ≈ - ∇L(z_tr) ⋅ ∇L(z_ts)`
  - Trong đó:
    - `∇L(z_tr)`: gradient loss tại mẫu train.
    - `∇L(z_ts)`: gradient loss tại mẫu test.
- Kết quả:
  - Một file JSON chứa các **điểm IF truyền thống** cho từng cặp (test, neighbor), dùng để:
    - Phân tích tương quan với DeltaF1 (hoặc DeltaLoss).
    - So sánh Baseline 1 với phương pháp IF* đề xuất.


## Đầu vào

### 1. Mô hình và thiết bị

- `device`:
  - Tự động chọn `cuda` nếu có GPU, ngược lại dùng `cpu`.

- Đường dẫn mô hình fine-tuned:
  - `final_model_path = Finetuned_Models/biobert-srl-best-model`.
  - Mô hình:
    - `AutoModelForTokenClassification` đã fine-tuned cho SRL.
  - Tokenizer:
    - `AutoTokenizer` tương ứng.

### 2. File k-NN đầu vào

- Danh sách `datasets_to_run_influence`:

  1. ParaVE Test (Baseline 1)
     - File k-NN:
       - `knn_results_weighted_k5_parave_test.json`
     - File influence đầu ra:
       - `influence_results_baseline1_traditional_parave_test.json`

  2. GramVar Test (Baseline 1)
     - File k-NN:
       - `knn_results_weighted_k5_gramvar_test.json`
     - File influence đầu ra:
       - `influence_results_baseline1_traditional_gramvar_test.json`

- Cấu trúc mỗi file k-NN (đã có từ bước trước):
  - Danh sách `results`, mỗi phần tử:
    - `test_sample`:
      - `sentence_text`
      - `verb_name`
      - `arg_label` (vd: ARG-0)
      - `matched_argument_text` (text của argument test)
    - `neighbors`:
      - Danh sách các láng giềng train k-NN:
        - `sentence_text`
        - `verb_name`
        - `arg_label`
        - Các thông tin thêm khác (score, file gốc, v.v.)

### 3. Dữ liệu gốc (corpus SRL)

- Thư mục train:
  - `train_data_dir_gramvar = Clean_Dataset/Corpus/Split_GramVar/Train`
  - `train_data_dir_parave = Clean_Dataset/Corpus/Split_ParaVE/Train`

- Thư mục test:
  - `test_data_dir_gramvar = Clean_Dataset/Corpus/Split_GramVar/Test`
  - `test_data_dir_parave = Clean_Dataset/Corpus/Split_ParaVE/Test`

- Mỗi file JSON trong các thư mục trên:
  - `verb_name_train_set.json` hoặc `verb_name_test_set.json`.
  - Cấu trúc mỗi phần tử:
    - `text`: câu gốc.
    - `predicate`: động từ trung tâm (predicate).
    - `arguments`: dict `ARG-x -> text`.
    - Các trường khác (nếu có).

### 4. Thư viện sử dụng

- `torch`, `torch.nn.functional as F`
- `os`, `glob`, `json`
- `tqdm.auto.tqdm`
- `transformers`:
  - `AutoTokenizer`
  - `AutoModelForTokenClassification`

## Đầu ra

1. File influence cho Baseline 1

   - `influence_results_baseline1_traditional_parave_test.json`
   - `influence_results_baseline1_traditional_gramvar_test.json`

2. Cấu trúc file output

   - Là danh sách các phần tử `result` giống cấu trúc k-NN đầu vào, nhưng trong từng `neighbor` có thêm:

     - `influence_score_traditional`:
       - Giá trị IF truyền thống:
         - `IF' = - grad_test ⋅ grad_train`.

   - Mỗi `result`:

     ```json
     {
       "test_sample": {
         "sentence_text": "...",
         "verb_name": "...",
         "arg_label": "ARG-0",
         "matched_argument_text": "..."
       },
       "neighbors": [
         {
           "sentence_text": "...",
           "verb_name": "...",
           "arg_label": "ARG-0",
           "score": 0.87,
           "source_dir": "...",
           "vector_file_path": "...",
           "influence_score_traditional": -0.12345
         },
         ...
       ]
     }
     ```

3. Log in ra màn hình

- Thông báo:
  - Tải mô hình, tokenizer thành công.
  - Tải dữ liệu train/test vào lookup.
  - Số lượng kết quả k-NN đã đọc.
  - Số lượng kết quả influence đã lưu cho mỗi bộ.

## Quy trình
### Bước 1: Tải mô hình, tokenizer, và cấu hình gradient

- Tải `tokenizer` và `model_ft` từ `final_model_path`.
- Đưa mô hình lên `device`.
- Đặt chế độ:
  - `model_ft.train()` để có thể tính gradient.
  - Nhưng với các module `Dropout` và `LayerNorm`:
    - Đặt `.eval()` để tránh ngẫu nhiên và giữ ổn định khi tính gradient.
- Lấy `label_to_id` từ:
  - `model_ft.config.label2id`.

### Bước 2: Tải dữ liệu thô vào lookup (train và test)

#### Hàm `load_raw_data_as_lookup(data_dirs)`

- Mục đích:
  - Đưa toàn bộ dữ liệu train/test vào một dict để tra cứu nhanh bằng `(text, verb_name)`.

- Các bước:
  1. Nhận một list `data_dirs` (train hoặc test).
  2. Với mỗi directory:
     - Lấy tất cả file JSON (vd: `begin_1_train_set.json`).
     - Tách `verb_name` từ tên file:
       - Loại bỏ `_train_set.json` hoặc `_test_set.json`.
     - Đọc JSON:
       - Với từng `item` trong file:
         - Tạo key:
           - `(item['text'], verb_name)`.
         - Lưu `item` vào dict:
           - `lookup[key] = item`.

- Kết quả:
  - `train_lookup`: chứa mọi mẫu train từ cả GramVar và ParaVE.
  - `test_lookup`: chứa mọi mẫu test từ cả GramVar và ParaVE.

### Bước 3: Token hóa và căn label cho một mẫu

#### Hàm `tokenize_and_align_labels_single(item, tokenizer, label_to_id)`

- Input:
  - `item`:
    - `text`, `predicate`, `arguments`.
- Các bước:
  1. Token hóa câu đầy đủ:
     - `tokenized_inputs = tokenizer(text, truncation=True, max_length=512, is_split_into_words=False)`.
  2. Lấy:
     - `word_ids = tokenized_inputs.word_ids(batch_index=0)`.
     - `tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs.input_ids)`.
  3. Khởi tạo:
     - `label_ids = [-100] * len(word_ids)`.
  4. Với mỗi argument `ARG-x`:
     - Bỏ qua nếu `arg_text` rỗng hoặc không có prefix `ARG`.
     - Token hóa `arg_text`:
       - `arg_tokens = tokenizer.tokenize(arg_text, add_special_tokens=False)`.
     - Quét `tokens` để tìm đoạn trùng khớp `arg_tokens`.
     - Đánh nhãn:
       - `B-predicate-ARG-x` cho token đầu.
       - `I-predicate-ARG-x` cho các token tiếp theo.
  5. Ánh xạ sang nhãn cuối cùng theo `word_ids`:
     - Nếu `word_idx is None`:
       - Gán `-100`.
     - Nếu `label_id != -100`:
       - Sử dụng label tương ứng.
     - Ngược lại:
       - Gán nhãn `O` (id từ `label_to_id['O']`).
  6. Đóng gói:
     - `tokenized_inputs['labels']` là tensor [1, seq_len].
     - `input_ids`, `attention_mask`, `token_type_ids` (nếu có) đều đưa về dạng tensor [1, seq_len].

- Output:
  - `tokenized_inputs` chứa cả input và labels cho một mẫu duy nhất.

### Bước 4: Tính loss và gradient dẹt cho một mẫu

#### Hàm `get_loss_and_flat_grad(item, model, tokenizer, label_to_id)`

- Mục đích:
  - Tính:
    - `loss` trên mẫu đó.
    - Gradient dẹt (flattened gradient) trên toàn bộ tham số mô hình.

- Các bước:
  1. `model.zero_grad()` để xóa gradient cũ.
  2. Gọi `tokenize_and_align_labels_single` để tạo inputs và labels.
  3. Đưa tất cả inputs về `device`.
  4. `outputs = model(**inputs, labels=labels)`.
  5. Lấy `loss = outputs.loss`.
  6. Gọi `loss.backward()` để tính gradient.
  7. Duyệt qua tất cả `param` trong `model.parameters()`:
     - Nếu `param.grad is not None`:
       - Lấy `param.grad.detach().clone().flatten()`.
       - Append vào list `grads`.
  8. Ghép tất cả gradient:
     - `flat_grad = torch.cat(grads)`.
  9. `model.zero_grad()` để reset.
  10. Trả về:
      - `loss.detach()`, `flat_grad`.

### Bước 5: Vòng lặp chính tính IF truyền thống

- Vòng lặp ngoài với từng bộ dữ liệu:
  - `name`, `knn_file`, `influence_file` trong `datasets_to_run_influence`.

#### Bước 5.1: Tải kết quả k-NN

- Đọc file `knn_file`:
  - `knn_results = json.load(f)`.
- Nếu file không tồn tại:
  - Báo lỗi và bỏ qua bộ dữ liệu đó.

#### Bước 5.2: Tính IF truyền thống cho từng cặp (test, neighbor)

- Khởi tạo:
  - `influence_results = []`.
  - `lookup_fail_count = 0`.

- Với mỗi `result` trong `knn_results`:
  1. Lấy thông tin mẫu test:
     - `test_sample_info = result['test_sample']`.
     - `test_key = (test_sample_info['sentence_text'], test_sample_info['verb_name'])`.
  2. Tìm `test_item` trong `test_lookup`:
     - Nếu không có:
       - Bỏ qua `result`.
  3. Tính gradient cho test:
     - `_loss_test, grad_test = get_loss_and_flat_grad(test_item, model_ft, tokenizer, label_to_id)`.
     - Nếu `grad_test is None`:
       - Bỏ qua `result`.
  4. Khởi tạo:
     - `processed_neighbors = []`.

  5. Với mỗi `neighbor` trong `result['neighbors']`:
     - Tạo `train_key = (neighbor['sentence_text'], neighbor['verb_name'])`.
     - Tìm `train_item` trong `train_lookup`.
     - Nếu không có:
       - Bỏ qua neighbor.
     - Tính gradient train:
       - `_loss_train, grad_train = get_loss_and_flat_grad(train_item, model_ft, tokenizer, label_to_id)`.
       - Nếu `grad_train is None`:
         - Bỏ qua neighbor.
     - Tính IF truyền thống:
       - `if_traditional_score = -torch.dot(grad_test, grad_train).item()`.
     - Gán:
       - `neighbor['influence_score_traditional'] = if_traditional_score`.
     - Thêm neighbor vào `processed_neighbors`.

  6. Gán:
     - `result['neighbors'] = processed_neighbors`.
  7. Append:
     - `influence_results.append(result)`.

#### Bước 5.3: Lưu file output

- Ghi `influence_results` vào:
  - `influence_file` tương ứng.

In [ ]:
# Import các thư viện cần thiết
import torch
import os
import json
import glob
from tqdm.auto import tqdm
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForTokenClassification

# --- 1. CẤU HÌNH ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# --- 2. THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'
output_dir = os.path.join(drive_base_path, 'matching_results')

# Đường dẫn đến mô hình fine-tuned
final_model_path = os.path.join(drive_base_path, 'Finetuned_Models/biobert-srl-best-model')

# Danh sách các bộ dữ liệu test cần chạy Influence (CHO BASELINE 1)
datasets_to_run_influence = [
    (
        "ParaVE Test (Baseline 1)", # Tên định danh
        os.path.join(output_dir, 'knn_results_weighted_k5_parave_test.json'), # File kNN đầu vào
        os.path.join(output_dir, 'influence_results_baseline1_traditional_parave_test.json') # File influence đầu ra
    ),
    (
        "GramVar Test (Baseline 1)",
        os.path.join(output_dir, 'knn_results_weighted_k5_gramvar_test.json'), # File kNN của GramVar
        os.path.join(output_dir, 'influence_results_baseline1_traditional_gramvar_test.json') # File influence đầu ra
    )
]

# Đường dẫn đến các file dữ liệu GỐC (để lấy thông tin đầy đủ)
train_data_dir_gramvar = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Train')
train_data_dir_parave = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Train')
test_data_dir_gramvar = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test')
test_data_dir_parave = os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test')

# --- 3. CÁC HÀM XỬ LÝ CỐT LÕI ---

def load_raw_data_as_lookup(data_dirs):
    """Tải tất cả dữ liệu thô từ các file json vào một dict để tra cứu nhanh."""
    print("Đang tải dữ liệu thô vào bộ nhớ đệm...")
    lookup = {}
    if not isinstance(data_dirs, list):
        data_dirs = [data_dirs]

    for data_dir in data_dirs:
        json_files = glob.glob(os.path.join(data_dir, '*.json'))
        for json_file in tqdm(json_files, desc=f"Tải {os.path.basename(data_dir)}", leave=False):
            try:
                base_name = os.path.basename(json_file)
                verb_name = "_".join(base_name.replace('_train_set.json', '').replace('_test_set.json', '').split('_'))

                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for item in data:
                        key = (item['text'], verb_name)
                        lookup[key] = item
            except Exception as e:
                print(f"Lỗi khi đọc file {json_file}: {e}")

    print(f" -> Đã tải {len(lookup)} mẫu.")
    return lookup

def tokenize_and_align_labels_single(item, tokenizer, label_to_id):
    """
    Tạo input_ids và labels cho MỘT mẫu duy nhất.
    """
    text = item['text']
    predicate = item['predicate']
    arguments = item['arguments']

    tokenized_inputs = tokenizer(text, truncation=True, max_length=512, is_split_into_words=False)
    word_ids = tokenized_inputs.word_ids(batch_index=0)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs.input_ids)
    label_ids = [-100] * len(word_ids)

    for arg_label, arg_text in arguments.items():
        if not arg_text or not arg_label.startswith('ARG'): continue
        arg_tokens = tokenizer.tokenize(arg_text, add_special_tokens=False)
        if not arg_tokens: continue

        for token_start_idx in range(len(tokens) - len(arg_tokens) + 1):
            if tokens[token_start_idx : token_start_idx + len(arg_tokens)] == arg_tokens:
                b_label_name = f"B-{predicate}-{arg_label}"
                if b_label_name in label_to_id:
                    label_ids[token_start_idx] = label_to_id[b_label_name]
                i_label_name = f"I-{predicate}-{arg_label}"
                if i_label_name in label_to_id:
                    for j in range(1, len(arg_tokens)):
                        label_ids[token_start_idx + j] = label_to_id[i_label_name]
                break

    final_labels = []
    for word_idx, label_id in zip(word_ids, label_ids):
        if word_idx is None:
            final_labels.append(-100)
        elif label_id != -100:
            final_labels.append(label_id)
        else:
            final_labels.append(label_to_id.get('O', 0))

    tokenized_inputs['labels'] = torch.tensor(final_labels).unsqueeze(0)
    tokenized_inputs['input_ids'] = torch.tensor(tokenized_inputs.input_ids).unsqueeze(0)
    tokenized_inputs['attention_mask'] = torch.tensor(tokenized_inputs.attention_mask).unsqueeze(0)
    if 'token_type_ids' in tokenized_inputs:
         tokenized_inputs['token_type_ids'] = torch.tensor(tokenized_inputs.token_type_ids).unsqueeze(0)

    return tokenized_inputs

def get_loss_and_flat_grad(item, model, tokenizer, label_to_id):
    """
    Tính toán Loss và Gradient dẹt (flattened gradient) cho một mẫu.
    """
    model.zero_grad()
    try:
        inputs = tokenize_and_align_labels_single(item, tokenizer, label_to_id)
        labels = inputs.pop('labels').to(device)
        inputs = {k: v.to(device) for k, v in inputs.items()}
    except Exception as e:
        return None, None

    with torch.enable_grad():
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss

    if loss is None: return None, None

    loss.backward()

    grads = []
    for param in model.parameters():
        if param.grad is not None:
            grads.append(param.grad.detach().clone().flatten())

    model.zero_grad()

    if not grads: return None, None

    flat_grad = torch.cat(grads)
    # *** THAY ĐỔI: Không cần trả về loss cho IF truyền thống ***
    return loss.detach(), flat_grad

# --- 4. THỰC THI CHƯƠNG TRÌNH ---
if __name__ == "__main__":

    # --- Bước 1: Tải mô hình và tokenizer  ---
    print("--- Bước 1: Tải mô hình Fine-tuned ---")
    try:
        tokenizer = AutoTokenizer.from_pretrained(final_model_path)
        model_ft = AutoModelForTokenClassification.from_pretrained(final_model_path).to(device)
        model_ft.train()
        for module in model_ft.modules():
            if isinstance(module, (torch.nn.Dropout, torch.nn.LayerNorm)):
                module.eval()
        label_to_id = model_ft.config.label2id
        print(" -> Tải mô hình và tokenizer thành công (ở chế độ train để lấy grad).")
    except Exception as e:
        print(f"LỖI: Không thể tải mô hình: {e}")
        exit()

    # --- Bước 2: Tải dữ liệu gốc (Tải 1 lần) ---
    print("\n--- Bước 2: Tải dữ liệu thô ---")
    train_lookup = load_raw_data_as_lookup([train_data_dir_gramvar, train_data_dir_parave])
    test_lookup = load_raw_data_as_lookup([test_data_dir_gramvar, test_data_dir_parave])

    # --- Bước 3: Vòng lặp xử lý từng bộ dữ liệu ---
    print("\n--- Bước 3: Bắt đầu tính toán Tầm ảnh hưởng (BASELINE 1 - IF TRUYỀN THỐNG) ---")

    for name, knn_file, influence_file in datasets_to_run_influence:

        print(f"\n=================================================")
        print(f"BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: {name}")
        print(f"=================================================")

        # 3a. Tải file kNN
        try:
            with open(knn_file, 'r', encoding='utf-8') as f:
                knn_results = json.load(f)
            print(f" -> Tải thành công {len(knn_results)} kết quả kNN từ {knn_file}.")
        except FileNotFoundError:
            print(f"LỖI: Không tìm thấy file kNN: {knn_file}. Bỏ qua bộ này.")
            continue # Chuyển sang bộ tiếp theo

        # 3b. Tính toán Influence
        influence_results = []
        lookup_fail_count = 0
        for result in tqdm(knn_results, desc=f"Tính Influence (Baseline 1) cho {name}"):
            test_sample_info = result['test_sample']
            test_key = (test_sample_info['sentence_text'], test_sample_info['verb_name'])
            test_item = test_lookup.get(test_key)
            if not test_item: continue

            # Không cần loss_test cho IF truyền thống
            _loss_test, grad_test = get_loss_and_flat_grad(test_item, model_ft, tokenizer, label_to_id)
            if grad_test is None: continue

            processed_neighbors = []
            for neighbor in result['neighbors']:
                train_key = (neighbor['sentence_text'], neighbor['verb_name'])
                train_item = train_lookup.get(train_key)
                if not train_item: continue

                # Không cần loss_train cho IF truyền thống
                _loss_train, grad_train = get_loss_and_flat_grad(train_item, model_ft, tokenizer, label_to_id)
                if grad_train is None: continue

                # TÍNH TOÁN IF TRUYỀN THỐNG (IF')
                # IF' = - (grad_test ⋅ grad_train)
                # Dấu trừ là vì loss.backward() tính gradient của loss,
                # và chúng ta muốn "ngược" lại (influence).
                # Theo slide 21, IF' xấp xỉ -∇L(z_tr) ⋅ ∇L(z_ts)
                if_traditional_score = -torch.dot(grad_test, grad_train).item()

                # Đổi tên key để file analysis nhận diện được
                neighbor['influence_score_traditional'] = if_traditional_score
                processed_neighbors.append(neighbor)

            result['neighbors'] = processed_neighbors
            influence_results.append(result)

        if lookup_fail_count > 0:
            print(f"CẢNH BÁO: Không thể tìm thấy {lookup_fail_count} mẫu (test hoặc train) trong 'lookup'.")

        # 3c. Lưu kết quả
        print(f"\n--- Lưu kết quả cho {name} ---")
        with open(influence_file, 'w', encoding='utf-8') as f:
            json.dump(influence_results, f, indent=4, ensure_ascii=False)
        print(f" -> Đã lưu thành công {len(influence_results)} kết quả vào: {influence_file}")

    print("\n--- QUÁ TRÌNH TÍNH TẦM ẢNH HƯỞNG (BASELINE 1) ĐÃ HOÀN TẤT! ---")


Sử dụng thiết bị: cuda
--- Bước 1: Tải mô hình Fine-tuned ---
 -> Tải mô hình và tokenizer thành công (ở chế độ train để lấy grad).

--- Bước 2: Tải dữ liệu thô ---
Đang tải dữ liệu thô vào bộ nhớ đệm...


Tải Train:   0%|          | 0/35 [00:00<?, ?it/s]

Tải Train:   0%|          | 0/35 [00:00<?, ?it/s]

 -> Đã tải 7327 mẫu.
Đang tải dữ liệu thô vào bộ nhớ đệm...


Tải Test:   0%|          | 0/35 [00:00<?, ?it/s]

...

 -> Đã tải 1185 mẫu.

--- Bước 3: Bắt đầu tính toán Tầm ảnh hưởng (BASELINE 1 - IF TRUYỀN THỐNG) ---

BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: ParaVE Test (Baseline 1)
 -> Tải thành công 699 kết quả kNN từ /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/matching_results/knn_results_weighted_k5_parave_test.json.


Tính Influence (Baseline 1) cho ParaVE Test (Baseline 1):   0%|          | 0/699 [00:00<?, ?it/s]


--- Lưu kết quả cho ParaVE Test (Baseline 1) ---
 -> Đã lưu thành công 699 kết quả vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/matching_results/influence_results_baseline1_traditional_parave_test.json

BẮT ĐẦU XỬ LÝ BỘ DỮ LIỆU: GramVar Test (Baseline 1)
 -> Tải thành công 1711 kết quả kNN từ /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/matching_results/knn_results_weighted_k5_gramvar_test.json.


Tính Influence (Baseline 1) cho GramVar Test (Baseline 1):   0%|          | 0/1711 [00:00<?, ?it/s]


--- Lưu kết quả cho GramVar Test (Baseline 1) ---
 -> Đã lưu thành công 1711 kết quả vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/matching_results/influence_results_baseline1_traditional_gramvar_test.json

--- QUÁ TRÌNH TÍNH TẦM ẢNH HƯỞNG (BASELINE 1) ĐÃ HOÀN TẤT! ---
